# Milestone 4 - Independent Project

## Author - Matthew Denko

## Instructions

    Milestone 4 is where everything you built in the previous milestones comes together. For this Milestone, you focus on improving model accuracy and summarizing your findings. Try explaining your findings as if you are presenting to your management team in layman’s terms. For example, talk about the influencing factors, what can be improved, what is important in your findings, what is the key aspect to focus on, what do the data tell them that they do not know.

    For Milestone 4 you should:

    (1) update Milestones 1 through 3, and assignment 9 based on feedback;
    (2) enhance your model results by trying different model and/or data enhancement techniques (Build 3 models with different enhancements and feature engineering techniques);
    (3) explain your choice of model and model accuracy; and
    (4) draw direct inferences and conclusions from model results (Describe how your model results can improve or provide a solution to the problem you have chosen).
    (5) Use graph and evidence from the data to prove your point. Part of being a data scientist is to tell a story that helps the business.

## Abstract

This dataset contains demographic data from the 1994 Census database which was gathered to see if it could predict if an Adult makes >50k annually.

## Goal of Running Model:

    I want to examine how demographic variables influence wealth metrics such as captial gains or income bracket. The goal of this analysis is to see if we can accurately predict someone's captial gain or whether or not they make greater than 50k annually. The results of this analysis could be used to identify members of society that might be more susceptible to making lower income or low capital gains. They could also be used to identify areas which are more likely to have high wealth metrics such as education and encourage people to explore these areas.

## How I Will Attempt To Improve My Results:

    I will attempt to improve the accuracy score and R2 value of my model by trying 3 different techniques:
    
    (1) I will add more features to my multiple linear regression model to see if they have better explanatory power. I will then use backwards selection to remove the features that do not add value. 
    
    (2) I will try a PCR model using the same list of features.
    
    (3) I will see if these variables are a better predictor of a different variable >50K, <=50k (whether or not someone makes more than 50 k anually)

In [ ]:
# Source Citation

source_citation = "Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science."
print("source citation = ",source_citation)
url = 'https://archive.ics.uci.edu/ml/datasets/Adult'
print("url =",url)

In [ ]:
# Load necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import *
import matplotlib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.feature_selection import RFE
import statsmodels.api as sms
from sklearn.cross_validation import cross_val_score
from sklearn.decomposition import PCA
import statsmodels.formula.api as sm
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
# Defining Functions

# Scale function

def scale(col):
    mean_col = np.mean(col)
    sd_col = np.std(col)
    std = (col - mean_col) / sd_col
    return std

In [ ]:
# Reading url

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
Adult= pd.read_csv(url, header=None)

# Assigning reasonable column names

Adult.columns = ["age","workclass","fnlwgt","education","educationnum","maritalstatus","occupation",
                 "relationship","race","sex","capitalgain","capitalloss","hoursperweek","nativecountry",">50K, <=50k"]
print(Adult.columns)
Adult.describe()

In [ ]:
## Drop Unecessary Columns

Adult = Adult.drop(columns=['fnlwgt', 'workclass', 'relationship','capitalloss','>50K, <=50k', 'education','race'])
print(Adult.columns)

# Data Cleanup

## Missing Data

In [ ]:
#Removing cases with missing data

Adult = Adult.replace(to_replace= "?", value=float("NaN"))
Adult_null = Adult.isnull().sum()
print(Adult_null)
print("There are 0 columns with missing data")

In [ ]:
## One Hot Encode Categorical Data

onehot_features = ['maritalstatus','occupation', 'sex','nativecountry']
dummy = pd.get_dummies(Adult[onehot_features])
dummy.head()
Adult = pd.concat([Adult,dummy],axis=1)
Adult = Adult.drop(columns=['maritalstatus','occupation','sex','nativecountry'])
print(Adult.columns)

# Modeling

## Multiple Linear Regression

In [ ]:
# Creating x an y columns

y_column = 'capitalgain'
x_columns = [x for x in Adult.columns if x not in y_column]

y = Adult[y_column]
x = Adult[x_columns]

In [ ]:
# Model initialization

regression_model = LinearRegression()

# Fit the data(train the model)

regression_model.fit(x, y)

# Predict

y_predicted = regression_model.predict(x)

#Summary Statistics

X = sms.add_constant(x)

# Note the diference in argument order

model = sms.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics

model.summary()

### Comments:
    
    The model is not very accurate, even with the increased number of features. The R2 value is at .30 and the adjusted R2 value is at .28 which both are higher than the previous model which had R2 and adjusted R2 values of .23. With the large amount of features I am worried about overfitting, so I will run backwards selection until I have the least amount of features with the highest adjusted R2 value.

In [ ]:
# 68 features - removing the first one

# Backwards Selection

model = LinearRegression() # use linear regression model for all features
rfe = RFE(model, 68) # use rfe
fit = rfe.fit(x, y) # fit our model
# Predict

y_predicted = fit.predict(x)

# model evaluation

rmse = mean_squared_error(y, y_predicted)
r2 = r2_score(y, y_predicted)
n = 32561
p = 68
adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % (fit.ranking_))
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)
print('Adj R2 score',adj_r2)

### Comments:

    Removing one feature did not affect the R2 score but did slightly increase the Adj R2 score. Which means that it reduced the issue of overfitting. I will now continue with removing an additional feature.

In [ ]:
# 67 features - removing the second one

# Backwards Selection

model = LinearRegression() # use linear regression model for all features
rfe = RFE(model, 67) # use rfe
fit = rfe.fit(x, y) # fit our model
# Predict

y_predicted = fit.predict(x)

# model evaluation

rmse = mean_squared_error(y, y_predicted)
r2 = r2_score(y, y_predicted)
n = 32561
p = 67
adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % (fit.ranking_))
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)
print('Adj R2 score',adj_r2)

### Comments:

    Removing another feature also slightly increased adjusted R2. I will continue with the backwards selection.

In [ ]:
# 66 features - removing the second one

# Backwards Selection

model = LinearRegression() # use linear regression model for all features
rfe = RFE(model, 66) # use rfe
fit = rfe.fit(x, y) # fit our model
# Predict

y_predicted = fit.predict(x)

# model evaluation

rmse = mean_squared_error(y, y_predicted)
r2 = r2_score(y, y_predicted)
n = 32561
p = 66
adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % (fit.ranking_))
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)
print('Adj R2 score',adj_r2)

### Comments:
    
    Adjusted Rsquared continues to increase. I will continue with backwards selection.

In [ ]:
# 65 features - removing the second one

# Backwards Selection

model = LinearRegression() # use linear regression model for all features
rfe = RFE(model, 65) # use rfe
fit = rfe.fit(x, y) # fit our model
# Predict

y_predicted = fit.predict(x)

# model evaluation

rmse = mean_squared_error(y, y_predicted)
r2 = r2_score(y, y_predicted)
n = 32561
p = 65
adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % (fit.ranking_))
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)
print('Adj R2 score',adj_r2)

### Comments:
    
    Adjusted Rsquared continues to increase. I will continue with backwards selection.

In [ ]:
# 64 features - removing the second one

# Backwards Selection

model = LinearRegression() # use linear regression model for all features
rfe = RFE(model, 64) # use rfe
fit = rfe.fit(x, y) # fit our model
# Predict

y_predicted = fit.predict(x)

# model evaluation

rmse = mean_squared_error(y, y_predicted)
r2 = r2_score(y, y_predicted)
n = 32561
p = 64
adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % (fit.ranking_))
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)
print('Adj R2 score',adj_r2)

### Comments:

    Adjusted Rsquared continues to increase. I will continue with backwards selection.

In [ ]:
# 63 features - removing the second one

# Backwards Selection

model = LinearRegression() # use linear regression model for all features
rfe = RFE(model, 63) # use rfe
fit = rfe.fit(x, y) # fit our model
# Predict

y_predicted = fit.predict(x)

# model evaluation

rmse = mean_squared_error(y, y_predicted)
r2 = r2_score(y, y_predicted)
n = 32561
p = 63
adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % (fit.ranking_))
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)
print('Adj R2 score',adj_r2)

### Comments:
    
    Adjusted Rsquared decreased, which means that I will use a final Multiple Linear Regression model with 64 features. The features that were removed were [maritalstatus_ Married-spouse-absent, maritalstatus_ Separated, nativecountry_ Philippines, nativecountry_ Laos, nativecountry_ Jamaica]. However, even with adding additional features to this model the R2 and adjusted R2 values are not very high. The variables in my model do not appear to have much sigificant explanatory power for capital-gain. I will next explore whether or not running a PCR model in the presence of additional features will have any sigificant uplift in R2/adjusted R2. However, if that method does not prove to work, I will see if the features are better suited explaining whether or not someone makes >50k annually.

## PCR

In [ ]:
#dropping features eliminated in backwards selection

Adult = Adult.drop(columns=['maritalstatus_ Married-spouse-absent', 'maritalstatus_ Separated', 
                             'nativecountry_ Philippines', 'nativecountry_ Laos', 'nativecountry_ Jamaica'])

In [ ]:
# Define the target and features:

target_label = 'capitalgain'
feature_labels = [x for x in Adult.columns if x not in [target_label]]

# One-hot encode inputs

Adult_expanded = pd.get_dummies(Adult, drop_first=True)
print('DataFrame one-hot-expanded shape: {}'.format(Adult_expanded.shape))

# Get target and original x-matrix

y = Adult[target_label]
x = Adult.as_matrix(columns=feature_labels)

In [ ]:
# Scale all columns

# Create x-scaled

x_scaled = np.apply_along_axis(scale, 0, x)
print(x_scaled)

# Create a scaled y-target.

y_scaled = np.apply_along_axis(scale, 0, y)
print(y_scaled)

In [ ]:
# PCA

pca = PCA(n_components=64)
pca_result = pca.fit_transform(x_scaled)
column_names = ['pc' + str(ix+1) for ix in range(x_scaled.shape[1])]
pca_df = pd.DataFrame(data = pca_result, columns=column_names)
pca_df[target_label] = y_scaled

In [ ]:
# Plot the explained variance for all principal components.

plt.plot(pca.explained_variance_ratio_)
plt.title('Explained variance by Principal Component Num')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance')

### Comments:

    The first 7 components appear to explain the majority of the variance, then there is a flat drop off of equivalent compentnets. The last 10 compenents do not explain much variance. Since the first 7 compenents appear to explore the majority of the variance, I will run a PCR with only the first 7 compenents.

In [ ]:
# 7 components

# Perform linear regression with the first N columns.

n = 7
formula_start = target_label + ' ~ '
formula_terms = ['pc' + str(x+1) for x in range(n)]
formula_end = ' + '.join(formula_terms)
formula_final = formula_start + formula_end
pcr_model = sm.ols(formula = formula_final, data=pca_df)
results = pcr_model.fit()

# Get most of the linear regression statistics we are interested in:

print(results.summary())

# Plot a histogram of the residuals

sns.distplot(results.resid, hist=True)
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Residual Histogram')

### Comments:

    The adjusted R-squared for the PCR model with 7 components is at .026 which is slightly less than the Multiple Linear Regression model. To ensure there is no underfitting I will run the model with 8 components. 

In [ ]:
# 8 components

# Perform linear regression with the first N columns.

n = 8
formula_start = target_label + ' ~ '
formula_terms = ['pc' + str(x+1) for x in range(n)]
formula_end = ' + '.join(formula_terms)
formula_final = formula_start + formula_end
pcr_model = sm.ols(formula = formula_final, data=pca_df)
results = pcr_model.fit()

# Get most of the linear regression statistics we are interested in:

print(results.summary())

# Plot a histogram of the residuals

sns.distplot(results.resid, hist=True)
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Residual Histogram')

### Comments:
    
    With 8 compenents the adjusted R2 score increases from .25 to .26. These implies that the previous model was underfitted using only the first 7 compenents. I will now test with 9 compenents to see if there is still underfitting.

In [ ]:
# 9 components

# Perform linear regression with the first N columns.

n = 9
formula_start = target_label + ' ~ '
formula_terms = ['pc' + str(x+1) for x in range(n)]
formula_end = ' + '.join(formula_terms)
formula_final = formula_start + formula_end
pcr_model = sm.ols(formula = formula_final, data=pca_df)
results = pcr_model.fit()

# Get most of the linear regression statistics we are interested in:

print(results.summary())

# Plot a histogram of the residuals

sns.distplot(results.resid, hist=True)
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Residual Histogram')

### Comments:
    
    With 9 compenents the adjusted R2 score goes back down to .25, which means that 8 components is the proper amount of components for this model. However, this score is lower than even the multiple linear regression model and we can infer that these variables are not good explainers of captial gain. I will now run a similar analyis using Naive Bayes but seeing if they are a good explaination of whether or not someone makes >50k anually.

## Naive Bayes

### Reload Dataset

In [ ]:
# Reading url

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
Adult= pd.read_csv(url, header=None)

# Assigning reasonable column names

Adult.columns = ["age","workclass","fnlwgt","education","educationnum","maritalstatus","occupation",
                 "relationship","race","sex","capitalgain","capitalloss","hoursperweek","nativecountry","k"]
print(Adult.columns)
Adult.describe()

#Removing cases with missing data

Adult = Adult.replace(to_replace= "?", value=float("NaN"))
Adult_null = Adult.isnull().sum()
print(Adult_null)
print("There are 0 columns with missing data")

### One Hot Encode

In [ ]:
## One Hot Encode Categorical Data

onehot_features = ['k']
dummy = pd.get_dummies(Adult[onehot_features])
dummy.head()
Adult = pd.concat([Adult,dummy],axis=1)
Adult = Adult.drop(columns=['k','k_ <=50K'])
print(Adult.columns)

### Create Model

In [ ]:
# Define the target and features:

target_label = 'k_ >50K'
non_features = ["workclass","fnlwgt","education","maritalstatus","occupation",
                 "relationship","race","sex","capitalloss","nativecountry","capitalgain"]
feature_labels = [x for x in Adult.columns if x not in [target_label] + non_features]

# Filter out non-features and non-targets

Adult = Adult.drop(non_features, axis=1)

# One-hot encode inputs

Adult_expanded = pd.get_dummies(Adult, drop_first=True)
print('DataFrame one-hot-expanded shape: {}'.format(Adult_expanded.shape))

# Get target and original x-matrix

y = Adult[target_label]
x = Adult.as_matrix(columns=feature_labels)

In [ ]:
Adult.columns

In [ ]:
# Split dataset into training set and test set

X_train, X_test, y_train, y_test = train_test_split(x, y, 
                                  test_size=0.3,random_state=42) # 70% training and 30% test

In [ ]:
# model

gnb = GaussianNB()

# train the model on the training sets only

gnb_model = gnb.fit(X_train, y_train)

In [ ]:
#Predict the response for test dataset

y_pred = gnb.predict(X_test)

### Evaluate Model

In [ ]:
# Model Accuracy

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
#print("R2 score:",metrics.r2_score(y_test, y_pred))

In [ ]:
# Model Accuracy: 10-fold cross-validation

ten_fold_scores = cross_val_score(GaussianNB(), x, y, scoring='accuracy', cv=10)
print (ten_fold_scores)
print ("The mean of the 10 fold cross validation scores is:", ten_fold_scores.mean())

### Comments:

    The accuracy score for this model is around 80% (which is fairly strong) and the 10 fold cross validation test has similar results. This model appears to be more accurate then the Multiple Linear Regression and PCR models ran trying to see if demographic variables can explain capital gain. This implies that age, education-num, and hours-per-week are fairly accurate predictors of whether or not an individual makes >50k. I ran a similar Naive Bayes model in assignment 4 which instead had capital gain as the target, that model had an accuracy score much lower (around .20). This is further evidence that other variables in this dataset are not strong predictors of capital gain. However, age, education-num, and hours-per-week are fairly strong indicators of whether or not someone makes >50k income.
    
    I conclude that:
    
    (1) age, education-num, and hours-per-week are fairly accurate indicators of whether or not someone makes >50k income, in this dataset
    
    (2) There are not strong demographic predictors in this dataset of capital gain